In [1]:
!pip install xlrd -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -U tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import datetime
import random
import time
from collections import Counter

import re



from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split

from imblearn.under_sampling import RandomUnderSampler 
from imblearn.datasets import make_imbalance

from tqdm import tqdm
import gc

import pickle




import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import *
nltk.download('punkt')


from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence


from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report



import keras
from keras import optimizers
from keras import backend as K
import tensorflow as tf
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional,Lambda
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from scipy.spatial.distance import cosine
from tensorflow.keras.losses import cosine_similarity


from keras.utils.data_utils import pad_sequences
from torch.nn.utils.rnn import pack_padded_sequence
import torch.nn.functional as F


import os
import random
import tensorflow as tf
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
ran=random.sample(range(1000, 100000), 10)

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)
    
    # Remove URL
    text = re.sub(r"http\S+", "", text)
    
    # Remove Hashtag
    text = re.sub(r"#\S+", "", text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove non character 
    text=re.sub(r'[\W_]+',' ', text)
    text=deEmojify(text)

    # Remove  number
    text = re.sub(" \d+", " ", text)

    text = re.sub("just", " ", text)
    text = re.sub("right", " ", text)
    '''


    output = re.sub(r'\s*[A-Za-z]+\b', '' , text)
    text = output.rstrip()

    persian = ['۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹'];
    for u in persian:
        text = re.sub(u, "", text);

  
    
    #
    #text = re.sub("کرونا", " ", text)
    #text = re.sub("ویروس", " ", text)
  
    '''
    return text


In [7]:
"""
Created on Tue Aug 16 14:43:05 2022

@author: behnam
"""

def flat_list(a):
    aa=[]
    for j in range(len(a)):
        
        aa=aa+a[j].split(',')
    return aa    
        
def text_preprocessing2(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '', text)
    
    # Remove URL
    text = re.sub(r"http\S+", "", text)
    
    # Remove Hashtag
    text = re.sub(r"#\S+", "", text)
    
    # Remove Hashtag
    text = re.sub(r"&\S+", "", text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    text=re.sub("[\(\[].*?[\)\]]", "", text)

    return text
    text = re.sub(r'\s+', ' ', text).strip()
    
    text=re.sub("[\(\[].*?[\)\]]", "", text)

    return text

In [8]:
pmfd=pd.read_excel('./drive/MyDrive/balanced_pmftc.xlsx')

In [60]:
ff='Authority'

In [61]:
foundation_annot_inClass=pmfd[pmfd['Label']==ff]
foundation_annot_outClass=pmfd[pmfd['Label']!=ff]
x_inClass=foundation_annot_inClass['Tweet']
x_outClass=foundation_annot_outClass['Tweet']
y_inClass=len(x_inClass)*[1]
y_outClass=len(x_outClass)*[0]
x=x_inClass.tolist()+x_outClass.tolist()
y=y_inClass+y_outClass

In [62]:
rus = RandomUnderSampler(random_state=42)
#rus = RandomUnderSampler(sampling_strategy={0: 500, 1: 500 },random_state=42)
x_res, y_res = rus.fit_resample(np.asarray(x).reshape(-1, 1),y)

print('Original dataset shape %s' % Counter(y))
print('Resampled dataset shape %s' % Counter(y_res))

x_res=x_res.reshape(len(x_res))

Original dataset shape Counter({0: 7500, 1: 500})
Resampled dataset shape Counter({0: 500, 1: 500})


Embedding layer

In [63]:
stop_words=[]
#with open('./drive/MyDrive/stop-words.txt') as file:
with open('./drive/MyDrive/Pesian_Stop_Words_List.txt') as file:  

    for line in file:
      stop_words.append(line.rstrip())

In [64]:
from sklearn.utils import shuffle

word_seq=[]
sw_removed_texts = []
for doc in tqdm(x_res):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words] 
  #sw_removed_texts.append(" ".join(filtered))
  word_seq.append(filtered)

100%|██████████| 1000/1000 [00:00<00:00, 3200.97it/s]


In [65]:
max_seq_len=120
MAX_NB_WORDS=100000

from sklearn.utils import shuffle

sw_removed_texts = []
for doc in tqdm(x_res):
  tokens = word_tokenize(doc)
  filtered = [word for word in tokens if word not in stop_words]
  sw_removed_texts.append(" ".join(filtered))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(sw_removed_texts)  
word_seq = tokenizer.texts_to_sequences(sw_removed_texts)

word_index = tokenizer.word_index
word_seq= pad_sequences(word_seq, maxlen=max_seq_len)



100%|██████████| 1000/1000 [00:00<00:00, 3110.15it/s]


In [66]:
word_seq

array([[   0,    0,    0, ...,  195, 1154, 2085],
       [   0,    0,    0, ..., 2090, 1156,   28],
       [   0,    0,    0, ...,    0, 2091, 1157],
       ...,
       [   0,    0,    0, ..., 6233, 6234,  219],
       [   0,    0,    0, ..., 6235, 1078,  123],
       [   0,    0,    0, ..., 6240,  715, 2083]], dtype=int32)

In [67]:
def set_reproducible(seed):
  

  os.environ['PYTHONHASHSEED'] = str(0)
  # For working on GPUs from "TensorFlow Determinism"
  os.environ['CUDA_VISBLE_DEVICE'] = ''
  #np.random.seed(seed)
  random.seed(seed)
  tf.random.set_seed(seed)



In [68]:
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embed_dim=64

def get_model():
  
  model = tf.keras.Sequential()

  model.add(Embedding(nb_words, embed_dim, input_length=max_seq_len,trainable=True))
  model.add(tf.keras.layers.LayerNormalization(axis=1)) 
  model.add(Bidirectional(LSTM(16, return_sequences= False)))
  #model.add(Bidirectional(LSTM(8, return_sequences= False)))
  model.add(Dense(64,activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(1,activation='sigmoid'))

  #model.summary()

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  es_callback = EarlyStopping(monitor='val_loss', patience=3)
  return model

In [69]:
'''
f_binary_all=[]
f_macro_all=[]
f_micro_all=[]
p_all =[]
r_all=[]
fold = 0


for k in ran:

  fold+=1
  x_train_a, x_test_a, y_train_a, y_test_a =train_test_split(word_seq,np.asanyarray(y_res), test_size=0.1, random_state=k)

  set_reproducible(k)
  model=get_model()
  es_callback = EarlyStopping(monitor='val_loss', patience=3)
  history = model.fit(x_train_a, y_train_a, batch_size=32, epochs=200,verbose=1, validation_split=0.2, callbacks=[es_callback], shuffle=True)

  yhat=model.predict(x_test_a, verbose=0)

  yp=[]
  for k in range(len(yhat)):
    if yhat[k][0]>0.45:
      yp.append(1)
    else:
      yp.append(0)

  print(classification_report(y_test_a,yp))   
  print('F1-score for fold: %d :' %fold)
  f1_macro=f1_score(y_test_a,yp,average='macro')
  f1_binary=f1_score(y_test_a,yp,average='binary')
  f1_micro=f1_score(y_test_a,yp,average='micro')

  print(f1_binary)
  print('-----') 
  f_binary_all.append(f1_binary)

  print(f1_macro)
  print('-----') 
  f_macro_all.append(f1_macro)

  print(f1_micro)
  print('-----') 
  f_micro_all.append(f1_micro)

  print('Recall-score for fold: %d :' %fold)
  r=recall_score(y_test_a,yp)
  print(r)
  print('-----')
  print('Precision-score for fold: %d :' %fold)
  p=precision_score(y_test_a,yp)
  print(p)
  print('-----')
  print('----------------------------------------------')

  r_all.append(r)
  p_all.append(p)
 

print("Average of F1_binary_Score = {}".format(np.mean(f_binary_all)))
print("Standard Deviation of F1_binary_Score = {}".format(np.std(f_binary_all)))
print('------')

print("Average of F1_macro_Score = {}".format(np.mean(f_macro_all)))
print("Standard Deviation of F1_macro_Score = {}".format(np.std(f_macro_all)))
print('------')

print("Average of F1_micro_Score = {}".format(np.mean(f_micro_all)))
print("Standard Deviation of F1_micro_Score = {}".format(np.std(f_micro_all)))
print('------')

print("Average of Recall_Score = {}".format(np.mean(r_all)))
print("Standard Deviation of Recall_Score = {}".format(np.std(r_all)))
print('------')

print("Average of Precision_Score = {}".format(np.mean(p_all)))
print("Standard Deviation of Precision_Score = {}".format(np.std(p_all)))
print('------')

'''

'\nf_binary_all=[]\nf_macro_all=[]\nf_micro_all=[]\np_all =[]\nr_all=[]\nfold = 0\n\n\nfor k in ran:\n\n  fold+=1\n  x_train_a, x_test_a, y_train_a, y_test_a =train_test_split(word_seq,np.asanyarray(y_res), test_size=0.1, random_state=k)\n\n  set_reproducible(k)\n  model=get_model()\n  es_callback = EarlyStopping(monitor=\'val_loss\', patience=3)\n  history = model.fit(x_train_a, y_train_a, batch_size=32, epochs=200,verbose=1, validation_split=0.2, callbacks=[es_callback], shuffle=True)\n\n  yhat=model.predict(x_test_a, verbose=0)\n\n  yp=[]\n  for k in range(len(yhat)):\n    if yhat[k][0]>0.45:\n      yp.append(1)\n    else:\n      yp.append(0)\n\n  print(classification_report(y_test_a,yp))   \n  print(\'F1-score for fold: %d :\' %fold)\n  f1_macro=f1_score(y_test_a,yp,average=\'macro\')\n  f1_binary=f1_score(y_test_a,yp,average=\'binary\')\n  f1_micro=f1_score(y_test_a,yp,average=\'micro\')\n\n  print(f1_binary)\n  print(\'-----\') \n  f_binary_all.append(f1_binary)\n\n  print(f1_mac

In [70]:

split_num=10
kf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=2022)

f_binary_all=[]
f_macro_all=[]
p_all =[]
r_all=[]
fold = 0

for train_idx, val_idx in tqdm(kf.split(word_seq,np.asarray(y_res))):

   

    x_train_f = word_seq[train_idx]
    y_train_f = np.asarray(y_res)[train_idx]
    x_val_f = word_seq[val_idx]
    y_val_f =np.asarray(y_res)[val_idx]

    set_reproducible(ran[fold])
    model=get_model()
    es_callback = EarlyStopping(monitor='val_loss', patience=10)
    model.fit(x_train_f, y_train_f, batch_size=16, epochs=200,verbose=1, callbacks=[es_callback],validation_data=(x_val_f, y_val_f), shuffle=False)
    #model.fit(x_train_f, y_train_f, batch_size=16, epochs=200,verbose=0,validation_data=(x_val_f, y_val_f), shuffle=False )

    #model.fit(x_train_f, y_train_f, batch_size=32, epochs=200,verbose=0, callbacks=[es_callback],validation_split=0.1)



    yhat=model.predict(x_val_f, verbose=0)

    yp=[]
    for k in range(len(yhat)):
      if yhat[k][0]>0.45:
        yp.append(1)
      else:
        yp.append(0)

    print(classification_report(y_val_f,yp)) 

    print('F1_binary-score for fold: %d :' %fold)
    f1_binary=f1_score(y_val_f,yp,average='binary')
    print(f1_binary)
    print('-----')

    print('F1_macro-score for fold: %d :' %fold)
    f1_macro=f1_score(y_val_f,yp,average='macro')
    print(f1_macro)
    print('-----')

    print('Recall-score for fold: %d :' %fold)
    r=recall_score(y_val_f,yp)
    print(r)
    print('-----')
    print('Precision-score for fold: %d :' %fold)
    p=precision_score(y_val_f,yp)
    print(p)
    print('-----')
    print('----------------------------------------------')
    f_binary_all.append(f1_binary)
    f_macro_all.append(f1_macro)
    r_all.append(r)
    p_all.append(p)
    fold+=1



print("Average of F1_binary_Score = {}".format(np.mean(f_binary_all)))
print("Standard Deviation of F1_binary_Score = {}".format(np.std(f_binary_all)))
print('------')

print("Average of F1_macro_Score = {}".format(np.mean(f_macro_all)))
print("Standard Deviation of F1_binary_Score = {}".format(np.std(f_macro_all)))
print('------')


print("Average of Recall_Score = {}".format(np.mean(r_all)))
print("Standard Deviation of Recall_Score = {}".format(np.std(r_all)))
print('------')

print("Average of Precision_Score = {}".format(np.mean(p_all)))
print("Standard Deviation of Precision_Score = {}".format(np.std(p_all)))
print('------')



print(np.mean([k for k  in f_macro_all if k>0.79 ]))
print(f_binary_all)



0it [00:00, ?it/s]

Epoch 1/200
57/57 [==============================] - 8s 67ms/step - loss: 1.1374 - accuracy: 0.5011 - val_loss: 0.6984 - val_accuracy: 0.5000
Epoch 2/200
57/57 [==============================] - 3s 51ms/step - loss: 0.8399 - accuracy: 0.4633 - val_loss: 0.6979 - val_accuracy: 0.5200
Epoch 3/200
57/57 [==============================] - 3s 52ms/step - loss: 0.7241 - accuracy: 0.4722 - val_loss: 0.6982 - val_accuracy: 0.5200
Epoch 4/200
57/57 [==============================] - 3s 52ms/step - loss: 0.5922 - accuracy: 0.7233 - val_loss: 0.7154 - val_accuracy: 0.5000
Epoch 5/200
57/57 [==============================] - 3s 52ms/step - loss: 0.3560 - accuracy: 0.9144 - val_loss: 1.0152 - val_accuracy: 0.4900
Epoch 6/200
57/57 [==============================] - 3s 52ms/step - loss: 0.1642 - accuracy: 0.9667 - val_loss: 1.0448 - val_accuracy: 0.5000
Epoch 7/200
57/57 [==============================] - 3s 53ms/step - loss: 0.0534 - accuracy: 0.9967 - val_loss: 0.8681 - val_accuracy: 0.6300
Epoch 

1it [00:46, 46.32s/it]

              precision    recall  f1-score   support

           0       0.76      0.50      0.60        50
           1       0.63      0.84      0.72        50

    accuracy                           0.67       100
   macro avg       0.69      0.67      0.66       100
weighted avg       0.69      0.67      0.66       100

F1_binary-score for fold: 0 :
0.7179487179487181
-----
F1_macro-score for fold: 0 :
0.6601791782514674
-----
Recall-score for fold: 0 :
0.84
-----
Precision-score for fold: 0 :
0.6268656716417911
-----
----------------------------------------------
Epoch 1/200
57/57 [==============================] - 10s 88ms/step - loss: 1.1569 - accuracy: 0.5489 - val_loss: 0.6956 - val_accuracy: 0.5000
Epoch 2/200
57/57 [==============================] - 4s 73ms/step - loss: 0.7615 - accuracy: 0.3578 - val_loss: 0.6980 - val_accuracy: 0.5000
Epoch 3/200
57/57 [==============================] - 3s 59ms/step - loss: 0.7534 - accuracy: 0.3722 - val_loss: 0.6885 - val_accuracy: 0.52

2it [01:35, 47.83s/it]

...

9it [10:43, 81.79s/it]

              precision    recall  f1-score   support

           0       0.66      0.74      0.70        50
           1       0.70      0.62      0.66        50

    accuracy                           0.68       100
   macro avg       0.68      0.68      0.68       100
weighted avg       0.68      0.68      0.68       100

F1_binary-score for fold: 8 :
0.6595744680851063
-----
F1_macro-score for fold: 8 :
0.6788438378161381
-----
Recall-score for fold: 8 :
0.62
-----
Precision-score for fold: 8 :
0.7045454545454546
-----
----------------------------------------------
Epoch 1/200
57/57 [==============================] - 10s 71ms/step - loss: 1.0271 - accuracy: 0.4644 - val_loss: 0.6802 - val_accuracy: 0.6100
Epoch 2/200
57/57 [==============================] - 3s 52ms/step - loss: 0.6776 - accuracy: 0.5589 - val_loss: 0.7670 - val_accuracy: 0.5000
Epoch 3/200
57/57 [==============================] - 3s 53ms/step - loss: 0.7633 - accuracy: 0.4211 - val_loss: 0.6796 - val_accuracy: 0.52

10it [11:35, 69.55s/it]

              precision    recall  f1-score   support

           0       0.66      0.62      0.64        50
           1       0.64      0.68      0.66        50

    accuracy                           0.65       100
   macro avg       0.65      0.65      0.65       100
weighted avg       0.65      0.65      0.65       100

F1_binary-score for fold: 9 :
0.6601941747572816
-----
F1_macro-score for fold: 9 :
0.6496847162446201
-----
Recall-score for fold: 9 :
0.68
-----
Precision-score for fold: 9 :
0.6415094339622641
-----
----------------------------------------------
Average of F1_binary_Score = 0.6834241880663805
Standard Deviation of F1_binary_Score = 0.037893857836442335
------
Average of F1_macro_Score = 0.681805840676603
Standard Deviation of F1_binary_Score = 0.03978741292698332
------
Average of Recall_Score = 0.6839999999999999
Standard Deviation of Recall_Score = 0.08890444308357147
------
Average of Precision_Score = 0.6954218763861506
Standard Deviation of Precision_Score 


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
